In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/watai-open-project-f23-intermediate/intermediate_test_dataset.npy


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds
import numpy as np

DATA_PATH = "/kaggle/working/data"
(train_ds,) = tfds.load(
   "stanford_online_products",
   split=["train"],
   shuffle_files=True,
   data_dir=DATA_PATH
)

# Data augmentation and normalization pipeline for training
train_pipeline = tf.keras.Sequential([
   # Add any augmentations before resizing
   tf.keras.layers.Resizing(height=64, width=64, crop_to_aspect_ratio=True),
   # Add any normalization after resizing
])

def preprocess(example):
   image = example["image"]
   label = example["super_class_id"]
   return (train_pipeline(image), label)
# This dataset is now ready to be used for batching + training with models

train_ds = train_ds.map(preprocess)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/59551 [00:00<?, ? examples/s]

Shuffling /kaggle/working/data/stanford_online_products/1.0.0.incompleteYX48S8/stanford_online_products-train.…

Generating test examples...:   0%|          | 0/60502 [00:00<?, ? examples/s]

Shuffling /kaggle/working/data/stanford_online_products/1.0.0.incompleteYX48S8/stanford_online_products-test.t…

Dataset stanford_online_products downloaded and prepared to /kaggle/working/data/stanford_online_products/1.0.0. Subsequent calls will reuse this data.


In [3]:
# Load pre-trained MobileNetV2 without top classification layers
base_model = tf.keras.applications.MobileNetV2(input_shape=(64, 64, 3),
                                               include_top=False,
                                               weights='imagenet')

# Freeze the pre-trained layers
base_model.trainable = False

9406464/9406464 [==============================] - 1s 0us/step


In [4]:
# Add custom classification layers on top of the pre-trained base
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(12, activation='softmax')  # 12 classes

model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])

In [5]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
# Train the model
model.fit(train_ds.batch(32), epochs=10)  # Adjust batch size as needed

Epoch 1/10
1861/1861 [==============================] - 53s 24ms/step - loss: 2.1304 - accuracy: 0.2838
Epoch 2/10
1861/1861 [==============================] - 45s 24ms/step - loss: 1.9865 - accuracy: 0.3330
Epoch 3/10
1861/1861 [==============================] - 45s 24ms/step - loss: 1.9502 - accuracy: 0.3449
Epoch 4/10
1861/1861 [==============================] - 45s 24ms/step - loss: 1.9300 - accuracy: 0.3503
Epoch 5/10
1861/1861 [==============================] - 45s 24ms/step - loss: 1.9195 - accuracy: 0.3552
Epoch 6/10
1861/1861 [==============================] - 45s 24ms/step - loss: 1.9094 - accuracy: 0.3594
Epoch 7/10
1861/1861 [==============================] - 45s 24ms/step - loss: 1.9053 - accuracy: 0.3589
Epoch 8/10
1861/1861 [==============================] - 45s 24ms/step - loss: 1.9011 - accuracy: 0.3608
Epoch 9/10
1861/1861 [==============================] - 45s 24ms/step - loss: 1.8995 - accuracy: 0.3619
Epoch 10/10
1861/1861 [==============================] - 46s 25m

In [7]:
# Load test dataset from the provided Numpy array and apply normalization
X_test = np.load("/kaggle/input/watai-open-project-f23-intermediate/intermediate_test_dataset.npy")  # Replace with your test data path
test_pipeline = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),  # Normalization
    tf.keras.layers.Resizing(height=224, width=224)
])

test_ds = tf.data.Dataset.from_tensor_slices(X_test).map(test_pipeline)

In [8]:
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)  # Get the index of the highest probability

N_test = X_test.shape[0]
df = pd.DataFrame({
    "Index": np.arange(N_test),
    "Label": predicted_labels  # Use the predicted labels
})
df.to_csv("submission.csv", index=False)

896/896 [==============================] - 6s 6ms/step
